In [1]:
# Notebook to evaluate active optics strategies

In [1]:
import numpy as np
import logging
import pickle
from scipy.linalg import block_diag
import scipy.io as spio
# required libraries for Reconstructor matrix computation or MPC
from scipy import sparse

import os.path
from os import path

from pathlib import Path
# import DOS library
import dos

from ruamel import yaml
from ruamel.yaml import YAML
y = YAML()
y.default_flow_style = None

sim_folder = 'dos/SH_AcO'
calibDataFolder = '/Users/rromano/Workspace/SIMCEO/calibration_dbs'

In [2]:
# Load data from yaml file

with open(Path(sim_folder)/'wfs48.yaml','r') as stream:
        wfs48_dt = yaml.safe_load(stream)
# with open(Path(sim_folder)/'AcO_controller.yaml','r') as stream:
#         aco_ctrl_dt = yaml.safe_load(stream)

# Calibrations data
#dt = np.load(Path(calibDataFolder)/'calib_dt_SH0p5fluxTH_46BM.npz', allow_pickle=True)
dt = np.load(Path(calibDataFolder)/'calib_dt_GSH0p5fluxTH_46BM.npz', allow_pickle=True)

In [3]:
def get_aco_recW(Dsh,n_bm,max_m1RBM,max_m2RBM):
    
    # --- W2 ---
    _U,sigma,VT = np.linalg.svd(Dsh,full_matrices=False)
    q = 40*sigma[-12:] #np.ones((12)) #np.zeros_like(sigma)
    W2 = VT[-12:,:].T.dot(np.diag(q)).dot(VT[-12:,:])

    # --- W3 ---
    rho_bm = 1.0 #0.1

    W_M1TxyzRxyz = 1/np.array(max_m1RBM)
    W_M2TxyzRxyz = 0.1/np.array(max_m2RBM)
    W_rbm_oa = block_diag(np.diag(W_M1TxyzRxyz), np.diag(W_M2TxyzRxyz))
    W_rbm_cs = block_diag(np.diag(W_M1TxyzRxyz[:-1]), np.diag(W_M2TxyzRxyz[:-1]))
    # Rescale RBM weighting matrix
    W_rbm_oa = (1/np.linalg.norm(W_rbm_oa))*W_rbm_oa
    W_rbm_cs = (1/np.linalg.norm(W_rbm_cs))*W_rbm_cs

    # OA segment weights
    W_m1oaBM = np.diag([1]*n_bm) #np.diag(1/s[:n_bm])
    W_m1oaBM =  rho_bm*(1/np.linalg.norm(W_m1oaBM))*W_m1oaBM
    # CS (center segment) weights
    W_m1csBM = np.diag([1]*n_bm) #np.diag(1/s[:n_bm])
    W_m1csBM =  rho_bm*(1/np.linalg.norm(W_m1csBM))*W_m1csBM

    W3_oa = block_diag(W_rbm_oa,W_m1oaBM)
    W3_cs = block_diag(W_rbm_cs,W_m1csBM)
    W3 = block_diag(np.kron(np.eye(6),W3_oa),W3_cs)
    sens_D = np.linalg.norm(Dsh, axis=0)**2
    W3 = W3.dot(np.diag(sens_D))
    W3 = (1/np.linalg.norm(W3))*W3
    
    return W2, W3



#### AcO reconstructor+controller settings

In [7]:
# *** Reconstructor tuning settings ***

# Number of bending modes
n_bm = 27

# Consolidate SH-WFS interaction matrices of all segments
Dsh = block_diag(*[Dseg[:,:12+n_bm] for Dseg in dt['calibdt'][()]['D'][:-1]])
Dsh = block_diag(Dsh,dt['calibdt'][()]['D'][-1][:,:10+n_bm])

# Max actuator command values
max_m1RBM = [3.0e-3,4.75e-3,4e-3] + [6.5e-4,5.75e-4,5.75e-4]
max_m2RBM = [3.0e-5,3.0e-5,3.0e-5] + [3.5e-3,3.5e-3,3.5e-3]
W2,W3 = get_aco_recW(Dsh,n_bm,max_m1RBM,max_m2RBM)

max_Fz = 147
K, rho_2, rho_3 = 0.25, 1.0, 10.0

# Print reconstructor regularization weights
np.set_printoptions(formatter={'float': '{: 0.6f}'.format})
w__, _w_,__w = np.split(np.diag(W3),[12,39])
print('OA BM weights:\n',_w_)
print('M1/2 RBM weights (M1/TxyzRxyz - M2/TxyzRxyz):\n',w__)
np.set_printoptions(formatter=None)


# *** Controller settings ****

# Plant dynamic model matrices
A = np.zeros((84+7*n_bm,84+7*n_bm))
B = np.identity(84+7*n_bm)
nx, nu = A.shape[0], B.shape[1]

# Prediction horizon
npred = 4
# Weighting matrices
Q = np.eye(nx)
R = 8*np.kron(np.eye(7),block_diag(np.diag(np.array([1]*6)),
                                   np.diag(np.array([1]*6)),
                                   np.diag(np.array([1]*n_bm))))
print(np.linalg.norm(Q),np.linalg.norm(R))

# Bending modes' data
Afz = {}

# OA segment weights
filepath = os.path.join('../ActiveOptics','Af_oa_axial_ver3.mat')
Afz['outer'] = spio.loadmat(filepath)['afprime']
_U,s_oa,V_oaT = np.linalg.svd(Afz['outer'],0)
bm2Fz_oa = np.dot(V_oaT.T, np.diag(1/s_oa)[:,:n_bm])
# CS (center segment) weights
filepath = os.path.join('../ActiveOptics','Af_cs154_axial.mat')
Afz['center'] = spio.loadmat(filepath)['afprime']
_U,s_cs,V_csT = np.linalg.svd(Afz['center'],0)
bm2Fz_cs = np.dot(V_csT.T, np.diag(1/s_cs)[:,:n_bm])

rbm_ranges = max_m1RBM+max_m2RBM
#oa_ranges = np.array(rbm_ranges + [max_Fz/np.linalg.norm(bm2Fz_oa)]*bm2Fz_oa.shape[0])
#cs_ranges = np.array(rbm_ranges + [max_Fz/np.linalg.norm(bm2Fz_cs)]*bm2Fz_cs.shape[0])
oa_ranges = np.array(rbm_ranges + [max_Fz]*bm2Fz_oa.shape[0])
cs_ranges = np.array(rbm_ranges + [max_Fz]*bm2Fz_cs.shape[0])

umax = np.hstack([np.kron(np.array([1]*6),oa_ranges),cs_ranges]) #   np.array([]) #  
umin = -umax
_Tu = block_diag(np.kron(np.eye(6),block_diag(np.eye(12),bm2Fz_oa)),
                 block_diag(np.eye(12),bm2Fz_cs))

# AcO dictionary
AcO_dt = {'SHAcO':{'D':Dsh,'n_bm':n_bm,'W2':W2,'W3':W3,
                 'K':K,'rho_2':rho_2,'rho_3':rho_3,
                 'wfsMask':dt['calibdt'][()]['wfsMask'],
                 'A':A,'B':B,
                 'Q':sparse.csc_matrix(Q),'R':sparse.csc_matrix(R),'npred':npred,
                 'umin':umin, 'umax':umax,'_Tu':_Tu,'J1_J3_ratio':10}}
# Pickles MPC data into string representation
with open(Path(sim_folder)/'SHAcO.pickle','wb') as f:
    pickle.dump(AcO_dt,f)
    

OA BM weights:
 [ 0.000004  0.000004  0.000007  0.000007  0.000007  0.000012  0.000013
  0.000011  0.000011  0.000019  0.000019  0.000015  0.000015  0.000020
  0.000020  0.000020  0.000026  0.000025  0.000028  0.000029  0.000024
  0.000025  0.000033  0.000032  0.000037  0.000037  0.000038]
M1/2 RBM weights (M1/TxyzRxyz - M2/TxyzRxyz):
 [ 0.000039  0.000023  0.000002  0.261524  0.279225  0.000006  0.000461
  0.000442  0.000022  0.000076  0.000072  0.000000]
16.522711641858304 132.18169313486644


#### Set initial M1/M2 states and generate respective yaml files

In [8]:
# Python code to set simulation states
from ruamel import yaml
from ruamel.yaml import YAML

# ----- Flag to control if simulation initial state is updated -----
updateX0 = False

test_case = 'random_RBM' #'BM_dist'

# Initialize states dictionary
if 'm1_x0_dt' not in locals():
    m1_x0_dt = {'mirror' : 'M1',
                'mirror_args': {'mirror_modes': 'bending modes','N_MODE': n_bm},
                'state': {'Txyz':[],'Rxyz':[],'modes':[]}}
if 'm2_x0_dt' not in locals():
    m2_x0_dt = {'mirror' : 'M2',
                'mirror_args': {},
                'state': {'Txyz':[],'Rxyz':[]}}

m1_x0_dt['state']['Txyz'] = np.zeros((7,3)).tolist()
m1_x0_dt['state']['Rxyz'] = np.zeros((7,3)).tolist()
m1_x0_dt['state']['modes'] = np.zeros((7,n_bm)).tolist()
m2_x0_dt['state']['Txyz'] = np.zeros((7,3)).tolist()
m2_x0_dt['state']['Rxyz'] =  np.zeros((7,3)).tolist()        
# Update number of M1 bending modes in yaml file
m1_x0_dt['mirror_args']['N_MODE'] = n_bm
        
if(test_case == 'fully_aligned'):
    pass

#        
elif(test_case == 'BM_dist'):
    m1_x0_dt['state']['modes'][0][2] = 5.0e-4
    m1_x0_dt['state']['modes'][0][1] = 5.0e-4
    
#    
elif(test_case == 'random_RBM') and updateX0:
    k_ = 3.0e-6
    as2urad = 4.84814
    x_rand = (2*np.random.rand(7,39)-1)
    # M1 random initial values
    x0_M1_Tx = k_*block_diag(np.kron(np.eye(6),0.057),1.654).dot(x_rand[:,0])
    x0_M1_Ty = k_*block_diag(np.kron(np.eye(6),0.06),0.99).dot(x_rand[:,1])
    x0_M1_Tz = k_*block_diag(np.kron(np.eye(6),0.144),0.201).dot(x_rand[:,2])
    m1_x0_dt['state']['Txyz'] = np.vstack([x0_M1_Tx,x0_M1_Ty,x0_M1_Tz]).T.tolist()
    x0_M1_Rx = k_*as2urad*block_diag(np.kron(np.eye(6),0.01),0.014).dot(x_rand[:,3])
    x0_M1_Ry = k_*as2urad*block_diag(np.kron(np.eye(6),0.004),0.016).dot(x_rand[:,4])
    x0_M1_Rz = k_*as2urad*block_diag(np.kron(np.eye(6),0.006),0.035).dot(x_rand[:,5])
    m1_x0_dt['state']['Rxyz'] = np.vstack([x0_M1_Rx,x0_M1_Ry,x0_M1_Rz]).T.tolist()
    #x0_BM = 0.0*k_states*4e-6*(2*np.random.rand(7,n_bm)-1)
    # M2 random initial values
    x0_M2_Tx = k_*block_diag(np.kron(np.eye(6),6.235),5.863).dot(x_rand[:,6])
    x0_M2_Ty = k_*block_diag(np.kron(np.eye(6),6.192),5.863).dot(x_rand[:,7])
    x0_M2_Tz = k_*block_diag(np.kron(np.eye(6),1.595),0.869).dot(x_rand[:,8])
    m2_x0_dt['state']['Txyz'] = np.vstack([x0_M2_Tx,x0_M2_Ty,x0_M2_Tz]).T.tolist()
    x0_M2_Rx = k_*as2urad*block_diag(np.kron(np.eye(6),0.474),0.463).dot(x_rand[:,9])
    x0_M2_Ry = k_*as2urad*block_diag(np.kron(np.eye(6),0.688),0.465).dot(x_rand[:,10])
    x0_M2_Rz = k_*as2urad*block_diag(np.kron(np.eye(6),2.315),2.186).dot(x_rand[:,11])
    m2_x0_dt['state']['Rxyz'] = np.vstack([x0_M2_Rx,x0_M2_Ry,x0_M2_Rz]).T.tolist()

if updateX0:
    # Update yaml files with M1/M2 initial conditions    
    with open(Path(sim_folder)/'M1.yaml','w') as f:
            y.dump(m1_x0_dt, f)
    with open(Path(sim_folder)/'M2.yaml','w') as f:
            y.dump(m2_x0_dt, f)
else:
    with open(Path(sim_folder)/'M1.yaml','r') as stream:
        m1_x0_dt = yaml.safe_load(stream)
    with open(Path(sim_folder)/'M2.yaml','r') as stream:
        m2_x0_dt = yaml.safe_load(stream)

In [9]:

# Set of guide star magnitudes to be investigated
mag_values = np.array([6, 8, 10, 12, 14, 16, 18, 20])

for imag in range(len(mag_values)): 
    # Set guide star magnitude
    wfs48_dt['source']['magnitude'] = (mag_values[imag]*np.array([1,1,1])).tolist()
    print('Magnitude:',wfs48_dt['source']['magnitude'])
    with open(Path(sim_folder)/'wfs48.yaml','w') as f:
        y.dump(wfs48_dt, f)

    # invoke DOS
    sim = dos.DOS(sim_folder,verbose=logging.WARNING)
    # Start simulation
    sim._run_()

    sim_log_dt = {'sim_log_dt':sim.logs.entries, 'AcO_dt':AcO_dt}
    # Pickles interaction matrix data into string representation
    filename = 'dt_mag%d_ctrlK%sreg20RecM.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    #filename = 'dt_mag%d_ctrlK%sTSVDRecM.pickle'% (mag_values[imag],str(K).replace(".", "p"))
    with open(Path('testDT')/filename,'wb') as f:
        pickle.dump(sim_log_dt,f)



INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


Magnitude: [6, 6, 6]
SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
Absolute constraints introduced to the MPC
-> J1:2.33e-13, J3:2.44e-15, ratio:9.56, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:2.33e-13 J1/(rho_3*J3):9.77

-> J1:2.44e-13, J3:7.99e-15, ratio:3.19, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:2.25e-13 J1/(rho_3*J3):6.05

-> J1:1.84e-13, J3:1.89e-14, ratio:3.19, ||s||:6.28e-09
27 BMs used in the reconstructor computation.
+> J1:1.68e-13 J1/(rho_3*J3):5.82

-> J1:1.35e-13, J3:4.36e-14, ratio:3.19, ||s||:2.67e-09
27 BMs used in the reconstructor computation.
+> J1:1.23e-13 J1/(rho_3*J3):5.93

-> J1:6.28e-14, J3:9.82e-14, ratio:2.06, ||s||:9.75e-10
27 BMs used in the reconstructor computation.
+> J1:5.46e-14 J1/(rho_3*J3):5.39

-> J1:2.04e-14, J3:2e-13, ratio:1.59, ||s||:3.2e-10
27 BMs used in the reconstructor computation.
+> J1:1.6e-14 J1/(rho_3*J3):4.17

-> J

INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
Absolute constraints introduced to the MPC
-> J1:2.34e-13, J3:2.44e-15, ratio:9.58, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:2.33e-13 J1/(rho_3*J3):9.78

-> J1:2.44e-13, J3:7.99e-15, ratio:3.19, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:2.25e-13 J1/(rho_3*J3):6.05

-> J1:1.84e-13, J3:1.89e-14, ratio:3.19, ||s||:6.28e-09
27 BMs used in the reconstructor computation.
+> J1:1.68e-13 J1/(rho_3*J3):5.83

-> J1:1.36e-13, J3:4.35e-14, ratio:3.2, ||s||:2.67e-09
27 BMs used in the reconstructor computation.
+> J1:1.24e-13 J1/(rho_3*J3):5.94

-> J1:6.31e-14, J3:9.79e-14, ratio:2.07, ||s||:9.75e-10
27 BMs used in the reconstructor computation.
+> J1:5.49e-14 J1/(rho_3*J3):5.41

-> J1:2.07e-14, J3:1.99e-13, ratio:1.61, ||s||:3.2e-10
27 BMs used in the reconstructor computation.
+> J1:1.64e-14 J1/(rho_3*J3):4.22

-> J1:5.9e-15, J3:4.14

INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


...

SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
Absolute constraints introduced to the MPC
-> J1:8.81e-13, J3:2.5e-15, ratio:35.3, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:8.93e-13 J1/(rho_3*J3):14.8

-> J1:9.2e-13, J3:6.4e-15, ratio:4.07, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:9.04e-13 J1/(rho_3*J3):8.82

-> J1:8.77e-13, J3:1.26e-14, ratio:4.86, ||s||:6.28e-09
27 BMs used in the reconstructor computation.
+> J1:8.67e-13 J1/(rho_3*J3):9.08

-> J1:8.23e-13, J3:1.84e-14, ratio:6.4, ||s||:2.67e-09
27 BMs used in the reconstructor computation.
+> J1:8.17e-13 J1/(rho_3*J3):9.24

-> J1:7.39e-13, J3:2.32e-14, ratio:7.13, ||s||:9.77e-10
27 BMs used in the reconstructor computation.
+> J1:7.34e-13 J1/(rho_3*J3):9.24

-> J1:7.04e-13, J3:2.7e-14, ratio:8.19, ||s||:3.21e-10
27 BMs used in the reconstructor computation.
+> J1:7.01e-13 J1/(rho_3*J3):9.52

-> J1:6.77e-13, J3:2.87e-

INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
Absolute constraints introduced to the MPC
-> J1:4.62e-12, J3:2.97e-15, ratio:156, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:4.65e-12 J1/(rho_3*J3):19.8

-> J1:4.63e-12, J3:5.88e-15, ratio:5.07, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:4.62e-12 J1/(rho_3*J3):9.79

-> J1:4.44e-12, J3:1.11e-14, ratio:5.09, ||s||:6.27e-09
27 BMs used in the reconstructor computation.
+> J1:4.43e-12 J1/(rho_3*J3):9.84

-> J1:4.25e-12, J3:1.54e-14, ratio:6.87, ||s||:2.68e-09
27 BMs used in the reconstructor computation.
+> J1:4.25e-12 J1/(rho_3*J3):9.9

-> J1:4.22e-12, J3:1.83e-14, ratio:8.36, ||s||:9.81e-10
27 BMs used in the reconstructor computation.
+> J1:4.22e-12 J1/(rho_3*J3):9.96

-> J1:4.19e-12, J3:2e-14, ratio:9.12, ||s||:3.24e-10
27 BMs used in the reconstructor computation.
+> J1:4.19e-12 J1/(rho_3*J3):9.98

-> J1:4.34e-12, J3:2.13e-

INFO:wfs48 data:Linked to data from wfs48
INFO:M1 Txyz:Linked to Txyz from M1
INFO:M1 Rxyz:Linked to Rxyz from M1
INFO:M2 Txyz:Linked to Txyz from M2
INFO:M2 Rxyz:Linked to Rxyz from M2
INFO:M1 BM:Linked to modes from M1


SIMCEO server received: Acknowledging connection from SIMCEO client!
27 BMs used in the reconstructor computation.
Absolute constraints introduced to the MPC
-> J1:2.75e-11, J3:4.55e-15, ratio:603, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:2.76e-11 J1/(rho_3*J3):31

-> J1:2.81e-11, J3:5.76e-15, ratio:8.09, ||s||:1.14e-08
27 BMs used in the reconstructor computation.
+> J1:2.81e-11 J1/(rho_3*J3):9.95

-> J1:2.74e-11, J3:1.08e-14, ratio:5.23, ||s||:6.28e-09
27 BMs used in the reconstructor computation.
+> J1:2.74e-11 J1/(rho_3*J3):9.91

-> J1:2.58e-11, J3:1.5e-14, ratio:6.74, ||s||:2.7e-09
27 BMs used in the reconstructor computation.
+> J1:2.58e-11 J1/(rho_3*J3):9.93

-> J1:2.66e-11, J3:1.78e-14, ratio:8.71, ||s||:1.01e-09
27 BMs used in the reconstructor computation.
+> J1:2.66e-11 J1/(rho_3*J3):9.99

-> J1:2.7e-11, J3:1.95e-14, ratio:9.22, ||s||:3.53e-10
27 BMs used in the reconstructor computation.
+> J1:2.7e-11 J1/(rho_3*J3):9.99

-> J1:2.69e-11, J3:2.06e-14